In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.chdir("/content/gdrive/MyDrive/JR_project")

In [3]:
import numpy as np
import pandas as pd
import gc
from tqdm.notebook import tqdm

# 机器学习
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import lightgbm as lgb


# 可视化
import matplotlib.pyplot as plt
import seaborn as sns

#https://www.kaggle.com/code/shun2741/baseline

In [9]:
#read csv file to dataframe
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')
order_products_train = pd.read_csv('order_products__train.csv')
order_products_prior = pd.read_csv('order_products__prior.csv')

特征生成

In [11]:
def add_features(df, orders_df, orders_product_prior_df):
    
    # 用于创建聚合功能的表
    order_mereged_df = pd.merge(orders_df, orders_product_prior_df, on='order_id', how='inner')
    
    # 用户×产品列
    order_mereged_df['user-product'] = order_mereged_df['user_id'].astype(str) + \
                                        "-" + \
                                        order_mereged_df['product_id'].astype(str) 
    
    # df 保存添加的功能
    feature_df = df.copy()
    
    # 用户×产品列
    feature_df['user-product'] = feature_df['user_id'].astype(str) + \
                                    "-" + \
                                    feature_df['product_id'].astype(str) 
    ######################### 按产品划分的特征 ###############################
    # 1-1.按产品划分的出现次数
    print("making product_count features")
    product_count_df = order_mereged_df.groupby("product_id").count()[['order_id']].reset_index().rename(
        columns={'order_id':'product_count'})
    
    # 添加您创建的功能
    feature_df = pd.merge(feature_df, product_count_df, on='product_id', how='left')
    
    # 删除中间生成的 df 并释放内存
    del product_count_df
    gc.collect() 

    # 1-2. 按产品划分的重新订单率
    print("making product_reordered_rate features")
    product_reordered_df = order_mereged_df.groupby("product_id").mean()[['reordered']].reset_index().rename(
        columns={'reordered':'product_reordered_rate'})
    
    # 添加您创建的功能
    feature_df = pd.merge(feature_df, product_reordered_df, on='product_id', how='left')
    
    # 删除中间生成的 df 并释放内存
    del product_reordered_df
    gc.collect() 

    ######################### 每个用户的特征 ###############################
    # 2-1.用户重新购买率
    print("making user_reorder_rate features")
    user_reorder_rate_df = order_mereged_df.groupby("user_id").mean()[["reordered"]].reset_index().rename(
        columns={'reordered':'user_reorder_rate'})
    
    # 添加您创建的功能
    feature_df = pd.merge(feature_df, user_reorder_rate_df, on='user_id', how='left')
    
    # 删除中间生成的 df 并释放内存
    del user_reorder_rate_df
    gc.collect()

    # 2-2. 用户过去的购买行为
    print("making user_action_mean features")
    action_list = ["order_number", "order_dow", "order_hour_of_day", "days_since_prior_order"]
    user_past_action_df = orders_df.groupby("user_id").mean()[action_list].reset_index().rename(
        columns={'order_number':'order_number_mean',
                 'order_dow':'order_dow_mean',
                 'order_hour_of_day':'order_hour_of_day_mean',
                 'days_since_prior_order':'days_since_prior_order_mean'})
    
    # 添加您创建的功能
    feature_df = pd.merge(feature_df, user_past_action_df, on='user_id', how='left')
    
    # 删除中间生成的 df 并释放内存
    del user_past_action_df
    gc.collect()

    # 2-3. 用户过去的订单数
    print("making user_action_count features")
    user_past_action_df = orders_df.groupby("user_id").count()['order_id'].reset_index().rename(
        columns={'order_id':'order_count'})
    
    # 作成した特徴を追加
    feature_df = pd.merge(feature_df, user_past_action_df, on='user_id', how='left')
    
    # 中間生成したdfを削除しメモリ解放
    del user_past_action_df
    gc.collect()
    
    ######################### 每个用户和产品的特点 ###############################
    # 3-1.用户是否已经重新订购了产品？
    print("making reordered features")
    reorder_product = order_mereged_df.query("reordered == 1")[['user_id', 'product_id']].drop_duplicates()
    reorder_product['reordered'] = 1

    # 添加您创建的功能
    feature_df = pd.merge(feature_df, reorder_product, on=['user_id', 'product_id'], how='left')
    
    # 补充在0，因为产品，从来没有重新订购成为缺陷处理
    feature_df['reordered'].fillna(0, inplace=True)
    
    # 中間生成したdfを削除しメモリ解放
    del reorder_product
    gc.collect()
    
    # 3-2. 该用户订购该商品的次数
    print("making user-product-count features")
    user_product_count_df = order_mereged_df.groupby("user-product").count()[["order_id"]].reset_index().rename(
        columns={'order_id':'user-product_count'})
    
    # 作成した特徴を追加
    feature_df = pd.merge(feature_df, user_product_count_df, on=['user-product'], how='left')
    
    # 中間生成したdfを削除しメモリ解放
    del user_product_count_df
    gc.collect()
    
    # 3-3. 用户在过去订单中订购目标产品的百分比
    feature_df['user_product_order_rate'] = feature_df['user-product_count'] / feature_df['order_count']
    
    ######################### 作成したdfを返す ###############################
    
    return feature_df

In [12]:
orders_prior = orders.query("eval_set == 'prior'")

向训练添加特征量

In [ ]:
train_feature_df = add_features(order_products_train, orders_prior, order_products_prior)

向训练添加特征量

In [ ]:
test_feature_df = add_features(test_df, orders_prior, orders_product_prior)

导出学习前数据

In [ ]:
train_feature_df.to_csv("train_feature.csv", header=True, index=False)
test_feature_df.to_csv("test_feature.csv", header=True, index=False)

In [ ]:
train_feature_df.head()

开始学习

根据所选模型的官方文档修改config中的参数

In [ ]:
# config
CFG = {
    
    "seed":0,
    "fold_num":5,
    
    
    "num_leaves":255, 
    "max_depth":-1, 
    "learning_rate":0.03, 
    "n_estimators":6000, 
    "subsample":0.6, 
    "subsample_freq":1, 
    "colsample_bytree":0.8, 
    "objective":'binary',
    "eval_metric": 'logloss',
    "early_stopping_rounds":50, 
    "verbose":100, 
    "step":20, 
    
    "debug":False 
}

In [ ]:
# 予測結果を管理する配列
answer = np.array([])
answer_train = np.array([])
AUC_list = []

# 学習用のデータを説明変数と目的変数に分割
if CFG['debug']:
    full_data_x = train_feature_df.drop(["order_id", "eval_set", "product_id", "target" , "user-product"],axis=1).iloc[:100000]
    full_data_y = train_feature_df[["target"]].iloc[:100000]
else:
    full_data_x = train_feature_df.drop(["order_id", "eval_set", "product_id", "target", "user-product"],axis=1)
    full_data_y = train_feature_df[["target"]]

# test用のデータ
test_x = test_feature_df.drop(["order_id", "eval_set", "product_id", "user_id", "user-product"],axis=1)

# GroupKFoldの分割
groups = full_data_x['user_id'].values
group_kfold = GroupKFold(n_splits=CFG['fold_num'])

for i, (train_index, valid_index) in enumerate(group_kfold.split(full_data_x, full_data_y, groups)):
    
    # デバッグモードの場合は1foldだけ実行して終了
    if CFG['debug'] and i == 1:
        break
    
    print("TRAIN:", train_index, "VALID:", valid_index)  
        
    # 学習と検証用にデータ分割
    train_x = full_data_x.iloc[train_index].drop(["user_id"],axis=1)
    valid_x = full_data_x.iloc[valid_index].drop(["user_id"],axis=1)
    
    train_y = full_data_y.iloc[train_index]
    valid_y = full_data_y.iloc[valid_index]
  
    # 回帰予測用のlightgbmモデルを設定
    gbm_model = lgb.LGBMClassifier(
        boosting_type='gbdt', # 木を作るときのルール
        num_leaves=CFG['num_leaves'], # 葉の数
        max_depth=CFG['max_depth'], # 最大の深さ
        learning_rate=CFG['learning_rate'], # 学習率
        n_estimators=CFG['n_estimators'], # 木を作る数
        subsample=CFG['subsample'], # 学習に用いるデータの割合
        subsample_freq=CFG['subsample_freq'], # サンプリングを行う頻度
        colsample_bytree=CFG['colsample_bytree'], # 学習に用いる列の割合
        objective=CFG['objective'], # 対象とするのは回帰問題
        random_state=CFG['seed'], # 乱数シード
        silent=False, # 学習内容の表示
        importance_type='gain' # 変数の重要度の計算方法
    )

    # モデルの学習
    gbm_model.fit(train_x, 
                  train_y,
                  eval_set=[(valid_x, valid_y)],
                  eval_metric=CFG["eval_metric"],
                  early_stopping_rounds=CFG['early_stopping_rounds'],
                  verbose=CFG['verbose'])
    
    # 予測
    gbm_pred = gbm_model.predict_proba(valid_x)
    
    # スコアの計算
    AUC = roc_auc_score(valid_y[['target']], gbm_pred[:,1])
    print("AUC : ", AUC)
    AUC_list += [AUC]

    # 予測
    temp_pred = gbm_model.predict_proba(test_x)
    
    # 答えの結果を記録
    if i == 0:
        answer = temp_pred[:, 1]
        answer_train = gbm_pred[:, 1]
    else:
        answer += temp_pred[:, 1]
    
    # デバッグモードの場合は１ループ目のみ色々可視化
    if CFG['debug'] and i == 0:

        # 説明変数の重要度を格納するためのデータフレームを作成
        feature_importances = pd.DataFrame()
        feature_importances['feature'] = train_x.columns
        feature_importances['importance'] = gbm_model.feature_importances_
    
        # 重要度が大きい順に可視化
        plt.figure(figsize=(16, 16))
        sns.barplot(data=feature_importances.sort_values('importance', ascending=False).head(50),
                    x='importance',
                    y='feature');
        plt.title('50 TOP feature importance')
        plt.show()
        
    # F1スコア
    best_F1 = 0
    best_thresfold = 0
    
    valid = pd.concat([full_data_x.iloc[valid_index], full_data_y.iloc[valid_index]], axis=1)
    
    for thresfold in range(0, CFG['step'], 1):
        if thresfold == 0:
            continue
        if thresfold > 6:
            break

        F1_sum = 0
        F1_num = valid['user_id'].nunique() # ユーザ数
        # その閾値での予測値
        valid['pred'] = np.where(gbm_pred < thresfold/CFG['step'], 0.0, 1.0)[:,1]
        
        # ユーザごとのF1スコアを計算し加算
        for data in tqdm(valid.groupby('user_id')):
            F1_sum += f1_score(data[1][['target']].values, data[1][['pred']].values) 
        
        # F1スコアの平均を算出
        F1 = F1_sum / F1_num 
        
        print("thresfold:", thresfold, " F1:", F1)
        
        if F1 > best_F1:
            best_F1 = F1
            best_thresfold = thresfold/CFG['step']
    print("bestF1", best_F1)
    print("best_thresfold", best_thresfold)
    # 使ったDFの削除
    del train_x, valid_x, train_y, valid_y, temp_pred, valid
    gc.collect()

# 提出用のファイルの作成
submission = test_feature_df.copy()
pred_test = answer / CFG['fold_num']

# 提出用ファイルの作成
submission['target'] = pred_test

In [ ]:
ans = pd.DataFrame()
for count, data in enumerate(tqdm(submission.groupby('order_id'))):
    
    order_dic = {}
    order_dic['order_id'] = str(int(data[1].iloc[0]['order_id']))
  
    first_flg = True
    t_str = ""
    for i in data[1].iterrows():
#         print(i[1]['target'])
        
        if i[1]['target'] > best_thresfold:
            if first_flg == True:
                t_str += str(i[1]['product_id'])
                first_flg = False
            else:
                t_str += " "
                t_str += str(i[1]['product_id'])

    if first_flg == True:
            order_dic['products'] = "None"
    else:
        order_dic['products'] = t_str
#     print(order_dic)
    ans = ans.append(order_dic, ignore_index=True)
    
#     if count > 10:
#         break
